In [1]:
!pip install -q kaggle

In [1]:
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"haydenji","key":"78a28bd346be35cfe15d7ea2c22d9975"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                      title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
iamsouravbanerjee/customer-shopping-trends-dataset       Customer Shopping Trends Dataset                146KB  2023-10-05 06:45:37          23142        478  1.0              
uom190346a/e-commerce-customer-behavior-dataset          E-commerce Customer Behavior Dataset              3KB  2023-11-10 03:01:13            945         25  1.0              
samyakb/student-stress-factors                           Student stress factors                           887B  2023-11-02 12:42:11           2979         61  0.9411765        
joebeachcapital/30000-spotify-songs                      30000 Spotify Songs                               3MB  202

In [4]:
!kaggle competitions download -c open-problems-single-cell-perturbations

100% 3.82G/3.83G [00:32<00:00, 183MB/s]
100% 3.83G/3.83G [00:32<00:00, 127MB/s]


In [5]:
!mkdir train
!unzip open-problems-single-cell-perturbations.zip -d train

Archive:  open-problems-single-cell-perturbations.zip
  inflating: train/adata_excluded_ids.csv  
  inflating: train/adata_obs_meta.csv  
  inflating: train/adata_train.parquet  
  inflating: train/de_train.parquet  
  inflating: train/id_map.csv        
  inflating: train/multiome_obs_meta.csv  
  inflating: train/multiome_train.parquet  
  inflating: train/multiome_var_meta.csv  
  inflating: train/sample_submission.csv  


In [6]:
import pandas as pd
import numpy as np

In [7]:
de_train = pd.read_parquet(f"./train/de_train.parquet")
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [8]:
ct = de_train["cell_type"].unique()
len(ct)

6

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
ct_df = de_train.iloc[:,0:1]

In [11]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(ct_df)
ct_ohe = ohe.transform(ct_df)
ct_ohe = ct_ohe.tolist()
de_train['cell_type_onehot'] = ct_ohe
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_onehot
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"


Now, let's try to represent chemical compound as molecular fingerprints:

In [12]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 15.9 MB/s eta 0:00:00


In [13]:
from rdkit import Chem
from rdkit.Chem import RDKFingerprint

In [14]:
sm_df = de_train["SMILES"]
print(len(sm_df))
sm_df = sm_df.dropna()
print(len(sm_df))
# sm_df = sm_df.unique()
# print(len(sm_df))

614
614


In [15]:
cs_arr = []
for sm in sm_df:
  try:
    cs = Chem.CanonSmiles(sm)
    cs_arr.append(cs)
  except:
    print('Invalid SMILES: ', sm)
print(len(cs_arr))

614


In [16]:
mols = [Chem.MolFromSmiles(x) for x in cs_arr]
fps = [np.array(RDKFingerprint(mol)) for mol in mols]
de_train['fingerprint'] = fps
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_onehot,fingerprint
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."


In [17]:
tmp = de_train.iloc[:,5:]
tmp.head()

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_onehot,fingerprint
0,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,1.043629,...,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,0.375550,...,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
2,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,0.214918,...,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
3,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,-0.212402,...,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,0.338703,...,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."


In [18]:
ge = tmp.iloc[:,:-2]
ge.head()

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,1.043629,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,0.375550,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,0.214918,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,-0.212402,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,0.338703,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [19]:
ct_sm = tmp.iloc[:,-2:]
ct_sm.head()

,cell_type_onehot,fingerprint
0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."


In [22]:
t1 = torch.tensor(ct_sm.iloc[0,0])
t2 = torch.tensor(ct_sm.iloc[0,1])
t = torch.cat((t1, t2), dim=0)
len(t)

2054

Now that we've converted both the cell type and compounds into DL-compatible formats, we can attempt to design a MLP that'll predict the gene expression profile based on some combination of these 2 variables:

In [21]:
# imports
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import time
import torch.nn.functional as F

In [23]:
class CustomDataset(Dataset):
    def __init__(self, df):
        self.dataframe = df
        self.ge = df.iloc[:,:-2]
        self.ct_sm = df.iloc[:,-2:]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        t1 = torch.tensor(self.ct_sm.iloc[0,0])
        t2 = torch.tensor(self.ct_sm.iloc[0,1])
        t = torch.cat((t1, t2), dim=0)
        return t, torch.Tensor(self.ge.iloc[index])

In [69]:
de_train_torch = CustomDataset(de_train.iloc[:,5:])
train_size = int(0.9 * len(de_train_torch))
val_size = int(len(de_train_torch) - train_size)
test_size = len(de_train_torch) - val_size - train_size
train_ds, val_ds = torch.utils.data.random_split(de_train_torch,
 [train_size, val_size])

In [40]:
print(train_size)

491


In [210]:
loss_fn = nn.MSELoss()
epochs = 500
learning_rate = 0.0000000000000000000000002
bsize = 300

In [70]:
train_loader = DataLoader(
    train_ds,
    batch_size=bsize,
    shuffle = True
)
val_loader = DataLoader(
    val_ds,
    batch_size=bsize,
    shuffle = True
)
# test_loader = DataLoader(
#     test_ds,
#     batch_size=bsize,
#     shuffle = False
# )

In [72]:
print(train_size)

552


In [71]:
# sanity check
for x, y in train_loader:
  print(x.shape)
  print(y.shape)
  break

torch.Size([300, 2054])
torch.Size([300, 18211])


In [30]:
from tqdm import tqdm

In [215]:
# mlp
class Net(nn.Module):
  def __init__(self, input_size, output_size):
    super(Net, self).__init__()
    self.dense1 = nn.Linear(input_size, 2 * input_size)
    self.bn1 = nn.BatchNorm1d(2 * input_size)
    self.do1 = nn.Dropout(0.9)
    self.dense2 = nn.Linear(2 * input_size, 4 * input_size)
    self.bn2 = nn.BatchNorm1d(4 * input_size)
    self.do2 = nn.Dropout(0.9)
    self.dense3 = nn.Linear(4 * input_size, 8 * input_size)
    self.bn3 = nn.BatchNorm1d(8 * input_size)
    self.dense4 = nn.Linear(8 * input_size, output_size)

  def forward(self, input_data):
    out = F.leaky_relu(self.bn1(self.dense1(input_data)))
    out = self.do1(out)
    out = F.leaky_relu(self.bn2(self.dense2(out)))
    out = self.do2(out)
    out = F.leaky_relu(self.bn3(self.dense3(out)))
    out = self.dense4(out)
    return out

In [32]:
# init weight
def init_weights(m):
  if isinstance(m, nn.Linear):
    # TODO: consider using other initialization schemes (He?)
    torch.nn.init.xavier_uniform_(m.weight.data)
    m.bias.data.fill_(0)

In [216]:
net = Net(input_size=2054, output_size=18211)

In [34]:
gpu_boole = torch.cuda.is_available()
print(gpu_boole)

True


In [217]:
if gpu_boole:
  net = net.cuda()

In [218]:
optim = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [180]:
def train(loader, model, optimizer, loss_fn, gpu_boole):
  model.train()
  total_loss = 0
  for data in tqdm(loader):
    x = data[0]
    y = data[1]
    if gpu_boole:
      x = x.cuda()
      y = y.cuda()
    optimizer.zero_grad()
    out = model(x)
    loss = loss_fn(out, y)
    total_loss += loss
    loss.backward()
    optimizer.step()
  # average loss per batch
  return total_loss / len(loader)

In [38]:
def evaluate(loader, model, loss_fn, gpu_boole):
  model.eval()
  total_loss = 0
  with torch.no_grad():
    for data in tqdm(loader):
      x = data[0]
      y = data[1]
      if gpu_boole:
        x = x.cuda()
        y = y.cuda()
      out = model(x)
      loss = loss_fn(out, y)
      total_loss += loss
  return total_loss / len(loader)

In [219]:
net.apply(init_weights)
loss = evaluate(val_loader, net, loss_fn, gpu_boole)
print("Average validation loss per batch: " + str(loss.cpu().data.numpy()))

100%|██████████| 1/1 [00:00<00:00,  3.65it/s]

Average validation loss per batch: 7.1486063


In [92]:
# training loop
#net.apply(init_weights)

train_loss = []
val_loss = []
min_val_loss = float('inf')

for epoch in range(epochs):
  time1 = time.time()
  print("Epoch",epoch+1,':')
  loss = train(train_loader, net, optim, loss_fn, gpu_boole)
  print("Average train loss per batch: " + str(loss.cpu().data.numpy()))
  train_loss.append(loss.cpu().data.numpy())
  loss = evaluate(val_loader, net, loss_fn, gpu_boole)
  print("Average validation loss per batch: " + str(loss.cpu().data.numpy()))
  val_loss.append(loss.cpu().data.numpy())
  if loss.cpu().data.numpy() < min_val_loss:
    torch.save(net.state_dict(), 'best_model_params.pt')

  time2 = time.time() #timekeeping
  print('\nElapsed time for epoch:',time2 - time1,'s')
  print('ETA of completion:',(time2 - time1)*(epochs - epoch - 1)/60,'minutes')
  print()

Epoch 1 :


100%|██████████| 2/2 [00:02<00:00,  1.27s/it]


Average train loss per batch: 12.768792


100%|██████████| 1/1 [00:00<00:00,  3.65it/s]


Average validation loss per batch: 14.361149

Elapsed time for epoch: 9.27456784248352 s
ETA of completion: 1.391185176372528 minutes

Epoch 2 :


...

Average validation loss per batch: 14.360924

Elapsed time for epoch: 9.42995572090149 s
ETA of completion: 0.1571659286816915 minutes

Epoch 10 :


100%|██████████| 2/2 [00:02<00:00,  1.24s/it]


Average train loss per batch: 12.957088


100%|██████████| 1/1 [00:00<00:00,  3.60it/s]


Average validation loss per batch: 14.360895

Elapsed time for epoch: 8.88849949836731 s
ETA of completion: 0.0 minutes



In [220]:
train_loss = []
val_loss = []
min_val_loss = float('inf')

for epoch in range(epochs):
  time1 = time.time()
  print("Epoch",epoch+1,':')
  loss = train(traisn_loader, net, optim, loss_fn, gpu_boole)
  print("Average train loss per batch: " + str(loss.cpu().data.numpy()))
  train_loss.append(loss.cpu().data.numpy())
  loss = evaluate(val_loader, net, loss_fn, gpu_boole)
  print("Average validation loss per batch: " + str(loss.cpu().data.numpy()))
  val_loss.append(loss.cpu().data.numpy())
  if loss.cpu().data.numpy() < min_val_loss:
    torch.save(net.state_dict(), 'best_model_params.pt')

  time2 = time.time() #timekeeping
  print('\nElapsed time for epoch:',time2 - time1,'s')
  print('ETA of completion:',(time2 - time1)*(epochs - epoch - 1)/60,'minutes')
  print()

Epoch 1 :


  0%|          | 0/2 [00:01<?, ?it/s]


OutOfMemoryError: ignored

In [98]:
de_train[de_train.cell_type == "Myeloid cells"].head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_onehot,fingerprint
9,Myeloid cells,Idelalisib,LSM-1205,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...,False,0.025146,0.316388,1.366885,1.301593,2.317619,...,0.794772,0.428973,0.605834,0.271988,0.492231,0.354721,1.471559,-0.259483,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, ..."
31,Myeloid cells,Crizotinib,LSM-1027,C[C@@H](Oc1cc(-c2cnn(C3CCNCC3)c2)cnc1N)c1c(Cl)...,False,1.322002,-0.083050,1.630350,0.238302,8.488463,...,-0.977755,-1.992494,1.265720,0.461353,-0.270218,-0.077017,-0.508337,0.454613,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, ..."
73,Myeloid cells,Linagliptin,LSM-45916,CC#CCn1c(N2CCC[C@@H](N)C2)nc2c1c(=O)n(Cc1nc(C)...,False,-0.463122,0.161972,0.446527,-0.244721,1.592027,...,0.301065,0.097764,0.016425,-0.050881,-0.414133,-0.259675,1.000058,-1.020959,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, ..."
95,Myeloid cells,Palbociclib,LSM-1071,CC(=O)c1c(C)c2cnc(Nc3ccc(N4CCNCC4)cn3)nc2n(C2C...,False,1.351549,-0.488389,0.235211,-0.534012,1.364028,...,0.119205,-0.002768,0.799264,-0.016142,-0.242718,0.546620,-0.099707,0.612288,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, ..."
105,Myeloid cells,Dabrafenib,LSM-6303,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...,True,0.254567,0.244074,-28.552191,-0.377238,-0.731186,...,0.010659,-1.890484,1.062484,-0.136040,-0.224016,-1.890099,-3.105304,-1.155821,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, ..."


In [94]:
id_map = pd.read_csv("./train/id_map.csv", sep=",", dtype=str)
id_map.columns
id_map.head()

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)
2,2,B cells,ABT737
3,3,B cells,AMD-070 (hydrochloride)
4,4,B cells,AT 7867


In [97]:
id_map.cell_type.value_counts()

B cells          128
Myeloid cells    127
Name: cell_type, dtype: int64

In [121]:
ohe_col = []
for x in id_map.cell_type:
  if x == "B cells":
    ohe_col.append([1.0, 0.0, 0.0, 0.0, 0.0, 0.0])
  else:
    ohe_col.append([0.0, 1.0, 0.0, 0.0, 0.0, 0.0])
id_map["ohe_col"] = ohe_col
id_map.head()

,id,cell_type,sm_name,ohe_col
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,1,B cells,ABT-199 (GDC-0199),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,2,B cells,ABT737,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,3,B cells,AMD-070 (hydrochloride),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,4,B cells,AT 7867,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [115]:
cp_df = de_train[["sm_name", "fingerprint"]]
cp_df["fingerprint"] = cp_df["fingerprint"].apply(tuple)
cp_df = cp_df.drop_duplicates()
len(cp_df)

<ipython-input-115-635315943ce1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cp_df["fingerprint"] = cp_df["fingerprint"].apply(tuple)


146

In [116]:
cp_df["fingerprint"] = cp_df["fingerprint"].apply(lambda x: np.array(x))
cp_df.head()

,sm_name,fingerprint
0,Clotrimazole,"[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,Mometasone Furoate,"[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."
8,Idelalisib,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, ..."
14,Vandetanib,"[1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, ..."
18,Bosutinib,"[1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, ..."


In [122]:
id_map_merged = pd.merge(id_map, cp_df, on="sm_name", how='inner')
id_map_merged.head()

,id,cell_type,sm_name,ohe_col,fingerprint
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, ..."
1,128,Myeloid cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, ..."
2,1,B cells,ABT-199 (GDC-0199),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
3,129,Myeloid cells,ABT-199 (GDC-0199),"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
4,2,B cells,ABT737,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."


In [140]:
genes = np.array(de_train.columns[5:18216])

In [198]:
out_fh = open("./submission.csv", 'w')
out_fh.write("id")
for g in genes:
  out_fh.write("," + g)
out_fh.write("\n")

1

In [197]:
net.eval()

Net(
  (dense1): Linear(in_features=2054, out_features=4108, bias=True)
  (bn1): BatchNorm1d(4108, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (do1): Dropout(p=0.7, inplace=False)
  (dense2): Linear(in_features=4108, out_features=8216, bias=True)
  (bn2): BatchNorm1d(8216, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (do2): Dropout(p=0.7, inplace=False)
  (dense3): Linear(in_features=8216, out_features=16432, bias=True)
  (bn3): BatchNorm1d(16432, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense4): Linear(in_features=16432, out_features=18211, bias=True)
)

In [199]:
for index, row in id_map_merged.iterrows():
  t1 = torch.tensor(row['ohe_col'])
  t2 = torch.tensor(row['fingerprint'])
  t = torch.cat((t1, t2), dim=0)
  if gpu_boole:
    t = t.cuda()
  t = t.reshape(1, -1)
  out = net(t)
  out_fh.write(str(index))
  out_np = out.cpu().data.numpy().flatten()
  for x in out_np:
    out_fh.write("," + str(x))
  out_fh.write("\n")

In [200]:
out_fh.close()

Latest submission had validation loss ~7.